# PROYECTO FINAL GRUPAL  - NBA-StatVision - Lesiones


### Importacion de librerias


In [26]:
import pandas as pd 
import numpy as np 


Importamos dataframe "play_by_play" pero solo leemos la primer fila que es el nombre de la columna


In [27]:
## creamos variable para alojar la ruta del archivo
ruta_archivo_csv = r'C:\Users\Fernando\Desktop\CSV de NBA\archive\csv\play_by_play.csv'

# df = pd.read_csv(ruta_archivo_csv)

## Leemos solo la primer fila con el nombre de las columnas.
def obtener_nombre_columnas(ruta_archivo_csv):
    
    columnas = pd.read_csv(ruta_archivo_csv, nrows=0).columns.tolist()
    return columnas




Vemos los nombres de las columnas de "play_by_play" (esto es porque nos trae la primer fila del dataset)

In [33]:
primera_fila = obtener_nombre_columnas(ruta_archivo_csv)
# mostramos las columnas:
print("Columnas del archivo 'play_by_play': ")
for col in primera_fila:
    print("-", col)

Columnas del archivo 'play_by_play': 
- game_id
- eventnum
- eventmsgtype
- eventmsgactiontype
- period
- wctimestring
- pctimestring
- homedescription
- neutraldescription
- visitordescription
- score
- scoremargin
- person1type
- player1_id
- player1_name
- player1_team_id
- player1_team_city
- player1_team_nickname
- player1_team_abbreviation
- person2type
- player2_id
- player2_name
- player2_team_id
- player2_team_city
- player2_team_nickname
- player2_team_abbreviation
- person3type
- player3_id
- player3_name
- player3_team_id
- player3_team_city
- player3_team_nickname
- player3_team_abbreviation
- video_available_flag


Este data set no tiene datos de fecha, solo 'game_id', entonces deberemos buscar en otro dataset, si hay game_id con fechas.

Buscamos en el dataset 'game_info' para ver si contiene una columna de fechas.

In [34]:
## Definimos la ruta donde se encuentra
ruta_game_info = r'C:\Users\Fernando\Desktop\CSV de NBA\archive\csv\game_info.csv' 

## Leemos el dataset
df = pd.read_csv(ruta_game_info)

## Lo mostramos 
df.head(2000)


,game_id,game_date,attendance,game_time
0,24600001,1946-11-01 00:00:00,NaN,NaN
1,24600003,1946-11-02 00:00:00,NaN,NaN
2,24600002,1946-11-02 00:00:00,NaN,NaN
3,24600004,1946-11-02 00:00:00,NaN,NaN
4,24600005,1946-11-02 00:00:00,NaN,NaN
...,...,...,...,...
1995,25200088,1952-12-05 00:00:00,NaN,NaN
1996,25200090,1952-12-06 00:00:00,NaN,NaN
1997,25200092,1952-12-06 00:00:00,NaN,NaN
1998,25200091,1952-12-06 00:00:00,NaN,NaN


Como tiene columna de fechas, vamos a filtrar los id que estén entre 2013 y 2023

In [37]:
# Cargamos nuevamente el data set, para "parsear" la columna 'game_date'.
df_games = pd.read_csv(ruta_game_info, parse_dates=['game_date'])

df_games['game_date'] = pd.to_datetime(df_games['game_date'])
# Filtrar por fechas:

games_filtred = df_games [
    (df_games['game_date'] >= '2012-10-30') &
    (df_games['game_date'] <= '2023-12-06')
]

# obtener el set de 'game_id' válidos
game_ids_validos = set(games_filtred['game_id'])
print(f'Cantidad de partidos entre 2013 y 2023: {len(game_ids_validos)}')

Cantidad de partidos entre 2013 y 2023: 12405


 Uso de Chunksize para filtrar el dataset de 'play_by_play' con los 'game_id' obtenidos de 'game_info'

In [38]:
# Creamos una lista para los Chunks

chunks = []

for chunk in pd.read_csv(r'C:\Users\Fernando\Desktop\CSV de NBA\archive\csv\play_by_play.csv', chunksize=100000):
    chunk_filtrado = chunk[chunk['game_id'].isin(game_ids_validos)]
    chunks.append(chunk_filtrado) ## llena la lista 

# Concatenamos los chunks
play_by_play_filtrado = pd.concat(chunks, ignore_index=True)

## mostramos el dataset filtrado
print(play_by_play_filtrado.shape)
play_by_play_filtrado.head()





(5228811, 34)


,game_id,eventnum,eventmsgtype,eventmsgactiontype,period,wctimestring,pctimestring,homedescription,neutraldescription,visitordescription,...,player2_team_nickname,player2_team_abbreviation,person3type,player3_id,player3_name,player3_team_id,player3_team_city,player3_team_nickname,player3_team_abbreviation,video_available_flag
0,41200161,0,12,0,1,5:41 PM,12:00,NaN,Start of 1st Period (5:41 PM EST),NaN,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
1,41200161,1,10,0,1,5:41 PM,12:00,Jump Ball Koufos vs. Bogut: Tip to Lawson,NaN,NaN,...,Warriors,GSW,4.0,201951,Ty Lawson,1.610613e+09,Denver,Nuggets,DEN,0
2,41200161,2,2,42,1,5:42 PM,11:40,MISS Fournier Driving Layup,NaN,Bogut BLOCK (1 BLK),...,NaN,NaN,5.0,101106,Andrew Bogut,1.610613e+09,Golden State,Warriors,GSW,0
3,41200161,3,4,0,1,5:42 PM,11:39,NaN,NaN,Bogut REBOUND (Off:0 Def:1),...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
4,41200161,4,2,1,1,5:42 PM,11:27,NaN,NaN,MISS Lee 15' Jump Shot,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0


Guardamos el dataset en un archivo nuevo de .CSV 


In [17]:
play_by_play_filtrado.to_csv(r'C:\Users\Fernando\Desktop\CSV de NBA\archive\csv\juego_por_juego.csv', index=False)
#Se guarda el archivo nuevo ya con los id de juego pertenecientes al rango de fecha que especificamos.


Visualizacion de la tabla juego_por_juego


In [18]:
## mostramos el dataset filtrado
print(play_by_play_filtrado.shape)
play_by_play_filtrado.head(1000)

(4940642, 34)


,game_id,eventnum,eventmsgtype,eventmsgactiontype,period,wctimestring,pctimestring,homedescription,neutraldescription,visitordescription,...,player2_team_nickname,player2_team_abbreviation,person3type,player3_id,player3_name,player3_team_id,player3_team_city,player3_team_nickname,player3_team_abbreviation,video_available_flag
0,41200161,0,12,0,1,5:41 PM,12:00,NaN,Start of 1st Period (5:41 PM EST),NaN,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
1,41200161,1,10,0,1,5:41 PM,12:00,Jump Ball Koufos vs. Bogut: Tip to Lawson,NaN,NaN,...,Warriors,GSW,4.0,201951,Ty Lawson,1.610613e+09,Denver,Nuggets,DEN,0
2,41200161,2,2,42,1,5:42 PM,11:40,MISS Fournier Driving Layup,NaN,Bogut BLOCK (1 BLK),...,NaN,NaN,5.0,101106,Andrew Bogut,1.610613e+09,Golden State,Warriors,GSW,0
3,41200161,3,4,0,1,5:42 PM,11:39,NaN,NaN,Bogut REBOUND (Off:0 Def:1),...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
4,41200161,4,2,1,1,5:42 PM,11:27,NaN,NaN,MISS Lee 15' Jump Shot,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,41200111,135,1,1,2,3:42 PM,8:56,Shumpert 24' 3PT Jump Shot (3 PTS) (Kidd 1 AST),NaN,NaN,...,Knicks,NYK,0.0,0,NaN,NaN,NaN,NaN,NaN,0
996,41200111,136,9,1,2,3:42 PM,8:54,NaN,NaN,Celtics Timeout: Regular (Reg.2 Short 0),...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
997,41200111,139,8,0,2,3:45 PM,8:54,NaN,NaN,SUB: Bradley FOR Crawford,...,Celtics,BOS,0.0,0,NaN,NaN,NaN,NaN,NaN,0
998,41200111,140,2,1,2,3:46 PM,8:31,Felton BLOCK (1 BLK),NaN,MISS Bradley 18' Jump Shot,...,NaN,NaN,4.0,101109,Raymond Felton,1.610613e+09,New York,Knicks,NYK,0


Observamos el tipo de datos:

In [10]:
for columna in play_by_play_filtrado.columns:
    print(columna)
    print(play_by_play_filtrado[columna].describe())
    print('------------------------------------')


game_id
count    4.940642e+06
mean     2.243871e+07
std      5.576207e+06
min      1.130000e+07
25%      2.150008e+07
50%      2.170085e+07
75%      2.200047e+07
max      4.210041e+07
Name: game_id, dtype: float64
------------------------------------
eventnum
count    4.940642e+06
mean     3.080306e+02
std      1.851543e+02
min      0.000000e+00
25%      1.500000e+02
50%      3.020000e+02
75%      4.550000e+02
max      1.018000e+03
Name: eventnum, dtype: float64
------------------------------------
eventmsgtype
count    4.940642e+06
mean     3.988490e+00
std      2.694526e+00
min      1.000000e+00
25%      2.000000e+00
50%      4.000000e+00
75%      5.000000e+00
max      1.800000e+01
Name: eventmsgtype, dtype: float64
------------------------------------
eventmsgactiontype
count    4.940642e+06
mean     1.428574e+01
std      2.737328e+01
min      0.000000e+00
25%      0.000000e+00
50%      1.000000e+00
75%      1.100000e+01
max      1.100000e+02
Name: eventmsgactiontype, dtype: float64

Abrimos Dataset de Lesiones_2013-2023

In [4]:
## creamos variable para alojar la ruta del archivo
ruta_archivo_lesionados_2013_2023 = r'C:\Users\Fernando\Desktop\CSV de NBA\Lesiones_2013-2023.csv'

df_lesionados_1323 = pd.read_csv(ruta_archivo_lesionados_2013_2023)

print(df_lesionados_1323.head())
print("Cantidad de filas y columnas: ", df_lesionados_1323.shape)

   PLAYER_ID    PLAYER_NAME SEASON  SEASON_NUM   AGE PLAYER_HEIGHT_INCHES  \
0    1630639    A.J. Lawson  22-23        22.5  22.0                   78   
1    1631260       AJ Green  22-23        22.5  23.0                   77   
2    1631100     AJ Griffin  22-23        22.5  19.0                   78   
3     203932   Aaron Gordon  22-23        22.5  27.0                   80   
4    1628988  Aaron Holiday  22-23        22.5  26.0                   72   

  PLAYER_WEIGHT  GP   MIN  USG_PCT  ...  AVG_SEC_PER_TOUCH  \
0           179  13   4.8    0.182  ...               1.39   
1           190  32   9.4    0.160  ...               2.61   
2           220  66  19.4    0.174  ...               2.08   
3           235  61  30.1    0.206  ...               2.94   
4           185  57  13.7    0.128  ...               4.17   

   AVG_DRIB_PER_TOUCH  ELBOW_TOUCHES  POST_TOUCHES  PAINT_TOUCHES     TEAM  \
0                0.62            0.0           0.0            0.2      NaN   
1       

In [6]:
for columna in df_lesionados_1323.columns:
    print(columna)
    print(df_lesionados_1323[columna].describe())
    print('------------------------------------')


PLAYER_ID
count    5.578000e+03
mean     7.789375e+05
std      7.189302e+05
min      7.080000e+02
25%      2.021300e+05
50%      2.035840e+05
75%      1.628399e+06
max      1.641645e+06
Name: PLAYER_ID, dtype: float64
------------------------------------
PLAYER_NAME
count             5578
unique            1389
top       Kemba Walker
freq                25
Name: PLAYER_NAME, dtype: object
------------------------------------
SEASON
count      5578
unique       10
top       21-22
freq        702
Name: SEASON, dtype: object
------------------------------------
SEASON_NUM
count    5578.000000
mean       18.241484
std         2.829290
min        13.500000
25%        15.500000
50%        18.500000
75%        20.500000
max        22.500000
Name: SEASON_NUM, dtype: float64
------------------------------------
AGE
count    5578.000000
mean       26.533883
std         4.253617
min        19.000000
25%        23.000000
50%        26.000000
75%        29.000000
max        43.000000
Name: AGE, dty

Abrimos el dataset Lesiones_2010-2020


In [7]:
## creamos variable para alojar la ruta del archivo
ruta_archivo_lesionados_2010_2020 = r'C:\Users\Fernando\Desktop\CSV de NBA\Lesiones_2010-2020.csv'

df_lesionados_1020 = pd.read_csv(ruta_archivo_lesionados_2010_2020)

print(df_lesionados_1020.head())
print("Cantidad de filas y columnas: ", df_lesionados_1020.shape)

       Date     Team         Nombre Acquired   Relinquished  \
0  03-10-10    Bulls  Carlos Boozer      NaN  Carlos Boozer   
1  06-10-10  Pistons  Jonas Jerebko      NaN  Jonas Jerebko   
2  06-10-10  Pistons  Terrico White      NaN  Terrico White   
3  08-10-10  Blazers     Jeff Ayres      NaN     Jeff Ayres   
4  08-10-10     Nets    Troy Murphy      NaN    Troy Murphy   

                                               Notes      ID  
0  fractured bone in right pinky finger (out inde...    2430  
1      torn right Achilles tendon (out indefinitely)  201973  
2  broken fifth metatarsal in right foot (out ind...  202358  
3          torn ACL in right knee (out indefinitely)  201965  
4             strained lower back (out indefinitely)    2211  
Cantidad de filas y columnas:  (27105, 7)


Vemos los tipos de datos: 

In [8]:
for columna in df_lesionados_1020.columns:
    print(columna)
    print(df_lesionados_1020[columna].describe())
    print('------------------------------------')

Date
count        27105
unique        2103
top       16-04-14
freq            76
Name: Date, dtype: object
------------------------------------
Team
count     27103
unique       32
top       Spurs
freq       1163
Name: Team, dtype: object
------------------------------------
Nombre
count          27101
unique          1158
top       Kevin Love
freq             139
Name: Nombre, dtype: object
------------------------------------
Acquired
count            9542
unique           1111
top       Dwyane Wade
freq               54
Name: Acquired, dtype: object
------------------------------------
Relinquished
count          17560
unique          1156
top       Kevin Love
freq             101
Name: Relinquished, dtype: object
------------------------------------
Notes
count                 27105
unique                 3114
top       activated from IL
freq                   7342
Name: Notes, dtype: object
------------------------------------
ID
count      27105
unique      1148
top       201567


Vamos a unir ambos dataset por nombre, debido a que una sola tabla tiene un numero id.

In [20]:
# Creamos una columna de referencia en df_1020 que combina Nombre y Relinquished
df_lesionados_1020['player_match'] = df_lesionados_1020['Nombre'].fillna(df_lesionados_1020['Relinquished'])
df_lesionados_1020['player_match'] = df_lesionados_1020['player_match'].str.strip().str.lower()  # Normalizamos
df_lesionados_1323['player_match'] = df_lesionados_1323['PLAYER_NAME'].str.strip().str.lower()   # Normalizamos

# Realizamos el Merge
df_lesiones_unido = pd.merge(df_lesionados_1020, df_lesionados_1323, on='player_match', how='outer', suffixes=('_1020','_1023'))

print(df_lesiones_unido)



            Date       Team        Nombre      Acquired  Relinquished  \
0       27-12-18      76ers         76ers           NaN         76ers   
1       26-10-16  Mavericks  A.J. Hammons           NaN  A.J. Hammons   
2       28-10-16  Mavericks  A.J. Hammons  A.J. Hammons           NaN   
3       30-10-16  Mavericks  A.J. Hammons           NaN  A.J. Hammons   
4       04-11-16  Mavericks  A.J. Hammons  A.J. Hammons           NaN   
...          ...        ...           ...           ...           ...   
157101       NaN        NaN           NaN           NaN           NaN   
157102  19-12-18      Bucks           NaN           NaN           NaN   
157103  16-03-19       Nets           NaN           NaN           NaN   
157104  31-12-19    Thunder           NaN           NaN           NaN   
157105  09-07-20    Wizards           NaN           NaN           NaN   

                                          Notes            ID    player_match  \
0                                  placed 

Vemos los tipos de datos:


In [23]:
for columna in df_lesiones_unido.columns:
    print(columna)
    print(df_lesiones_unido[columna].describe())
    print('------------------------------------')

Date
count       156056
unique        2103
top       12-04-17
freq           560
Name: Date, dtype: object
------------------------------------
Team
count     156052
unique        32
top        Spurs
freq        6930
Name: Team, dtype: object
------------------------------------
Nombre
count           156052
unique            1158
top       Derrick Rose
freq              2852
Name: Nombre, dtype: object
------------------------------------
Acquired
count            56345
unique            1111
top       Derrick Rose
freq               897
Name: Acquired, dtype: object
------------------------------------
Relinquished
count            99713
unique            1156
top       Derrick Rose
freq              1955
Name: Relinquished, dtype: object
------------------------------------
Notes
count                156056
unique                 3114
top       activated from IL
freq                  40238
Name: Notes, dtype: object
------------------------------------
ID
count     156056
unique    

Traigo las columnas que me interesan.

In [22]:
print(df_lesiones_unido[['player_match', 'Date', 'TEAM', 'INJURED ON', 'Notes']].head(50))

    player_match      Date TEAM INJURED ON                       Notes
0          76ers  27-12-18  NaN        NaN                placed on IL
1   a.j. hammons  26-10-16  NaN        NaN                placed on IL
2   a.j. hammons  28-10-16  NaN        NaN           activated from IL
3   a.j. hammons  30-10-16  NaN        NaN                placed on IL
4   a.j. hammons  04-11-16  NaN        NaN           activated from IL
5   a.j. hammons  27-12-16  NaN        NaN                placed on IL
6   a.j. hammons  29-12-16  NaN        NaN           activated from IL
7   a.j. hammons  03-01-17  NaN        NaN                placed on IL
8   a.j. hammons  26-01-17  NaN        NaN           activated from IL
9   a.j. hammons  24-02-17  NaN        NaN                 "activated"
10  a.j. hammons  27-02-17  NaN        NaN                placed on IL
11  a.j. hammons  01-03-17  NaN        NaN           activated from IL
12  a.j. hammons  05-03-17  NaN        NaN                placed on IL
13  a.

In [25]:
print("Cantidad de filas y columnas: ", df_lesiones_unido.shape)

Cantidad de filas y columnas:  (157106, 40)
